In [ ]:
from __future__ import print_function

In [ ]:
training_data = [
    ['Green' , 3 , 'Apple'],
    ['Yellow' ,3 ,'Apple'],
    ['Red' ,1 , 'Grape'],
    ['Red' ,1 ,'Grape'],
    ['Yellow' ,3 ,'Lemon']
]

In [ ]:
header = ['color' ,'Diameter' ,'Label']

In [ ]:
def unique_values(rows ,col):
  return set([row[col] for row in rows])

In [ ]:
unique_values(training_data ,0)

{'Green', 'Red', 'Yellow'}

In [ ]:
def class_counts(rows):
  counts = {}
  for row in rows:
    label= row[-1]
    if label not in counts:
      counts[label] = 0
    counts[label] += 1
  return counts

# def class_counts(rows):
#     """Counts the number of each type of example in a dataset."""
#     counts = {}  # a dictionary of label -> count.
#     for row in rows:
#         # in our dataset format, the label is always the last column
#         label = row[-1]
#         if label not in counts:
#             counts[label] = 0
#         counts[label] += 1
#     return counts

In [ ]:
def class_counts1(rows):
  counts = {}
  for row in rows:
    label= row[-3]
  return label

In [ ]:
class_counts1(training_data)

'Yellow'

In [ ]:
for i in training_data:
  lab = i[-1]
  print(lab)

Apple
Apple
Grape
Grape
Lemon


In [ ]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [ ]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [ ]:
def is_numeric(value):
  return isinstance(value ,int) or isinstance(value,float)
is_numeric(7)

True

In [ ]:
class Question:
  def __init__(self,column,value):
    self.column = column
    self.value = value
  def match(self,example):
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value
  def __repr__(self):
    condition = "=="
    if is_numeric(self.value):
      condition = ">="
    return "Is %s %s %s?" %(
        header[self.column] , condition, str(self.value))



In [ ]:
training_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [ ]:
Question(2,0)

Is Label >= 0?

In [ ]:
q = Question(0 ,'Green')

In [ ]:
example = training_data[0]

In [ ]:
q.match(example)

True

In [ ]:
def partition(rows , question):
  true_rows ,false_rows = [],[]
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
  return true_rows ,false_rows

In [ ]:
true_rows ,false_rows = partition(training_data, Question(0,'Red'))
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [ ]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [ ]:
# def gini(rows):
#   counts = class_count(rows)
#   impurity = 1
#   for lbl in counts:
#     prob_of_lbl = counts[lbl]/float(len(rows))
#     impurity -= prob_of_lbl**2
#   return impurity


def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [ ]:
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [ ]:
some_mixing = [['Apple'],['Orange']]
gini(some_mixing)

0.5

In [ ]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]

gini(lots_of_mixing)

0.7999999999999998

In [ ]:
def info_gain(left ,right,current_uncertainity):
  p = float(len(left)/len(left)+len(right))
  return current_uncertainity - p*gini(left) - (1-p) * gini(right)


In [ ]:
current_uncertainity = gini(training_data)
current_uncertainity

0.6399999999999999

In [ ]:
# true_rows ,false_rows = partition(training_data , Question(0,'Green'))
# info_gain(true_rows ,false_rows,current_uncertainity)

# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainity)

3.1399999999999997

In [ ]:
true_rows , false_rows = partition(training_data , Question(0,'Red'))
info_gain(true_rows ,false_rows,current_uncertainity)

1.9733333333333334

In [ ]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [ ]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [ ]:
def find_best_split(rows):
  best_gain = 0
  best_question = None
  current_uncertainity = gini(rows)
  n_features = len(rows[0]) - 1

  for col in range(n_features):
    values = set([row[col] for row in rows])

    for val in values:
      question = Question(col ,val)
      true_rows ,false_rows = partition(rows ,question)

      if len(true_rows) == 0 or len(false_rows) == 0:
        continue

      gain = info_gain(true_rows ,false_rows ,current_uncertainity)

      if gain >= best_gain:
        best_gain ,best_question = gain ,question
  return best_gain ,best_question



In [ ]:
find_best_split(training_data)

(3.1399999999999997, Is color == Green?)

In [ ]:
class Leaf:
  def __init__(self ,rows):
    self.predictions = class_counts(rows)


In [ ]:
class Decision_node:
  def __init__(self,question , true_branch,false_branch):
    self.question = question
    self.true_branch =true_branch
    self.false_branch = false_branch


In [ ]:
def build_tree(rows):
  gain,question = find_best_split(rows)
  if gain == 0:
    return Leaf(rows)
  true_rows,false_rows  = partition(rows , question)
  true_branch = build_tree(true_rows)
  false_branch = build_tree(false_rows)
  return Decision_node(question ,true_branch , false_branch)


In [ ]:
def print_tree(node, spacing = ""):
  if isinstance(node, Leaf):
    print(spacing ,"+Predict" ,node.predictions)
    return
  print(spacing +str(node.question))
  print(spacing + '--> True')
  print_tree(node.true_branch, spacing + "  ")
  print(spacing + '--> False')
  print_tree(node.false_branch ,spacing + " ")


In [ ]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is color == Green?
--> True
   +Predict {'Apple': 1}
--> False
 Is color == Red?
 --> True
    +Predict {'Grape': 2}
 --> False
   +Predict {'Apple': 1, 'Lemon': 1}


In [ ]:
def classify(row ,node):
  if isinstance(node,Leaf):
    return node.predictions
  if node.question.match(row):
    return classify(row,node.true_branch)
  else:
    return classify(row ,node.false_branch)


In [ ]:
classify(training_data[0] ,my_tree)

{'Apple': 1}

In [ ]:
def print_leaf(counts):
  total = sum(counts.values())*1.0
  probs = {}
  for lbl in counts.keys():
    probs[lbl] = str(int(counts[lbl]/total*100)) + "%"
  return probs

In [ ]:
print_leaf(classify(training_data[0] ,my_tree))

{'Apple': '100%'}

In [ ]:
print_leaf(classify(training_data[1],my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [ ]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [ ]:
for row in testing_data:
  print('Actual: %s. predicted: %s' %
        (row[-1],print_leaf(classify(row,my_tree))))

Actual: Apple. predicted: {'Apple': '100%'}
Actual: Apple. predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. predicted: {'Grape': '100%'}
Actual: Grape. predicted: {'Grape': '100%'}
Actual: Lemon. predicted: {'Apple': '50%', 'Lemon': '50%'}
